In [1]:
import os
import torch
from torch import nn
from PIL import Image
import torch.nn.functional as F
import matplotlib.pyplot as plt

from clipseg_model import CLIPSeg

%load_ext autoreload
%autoreload 2

# Dataset

## CUB Parts

In [ ]:
from data.cub_parts_dataset import CUBPartsDataset

dataset_train = CUBPartsDataset('datasets/PASCUB', split='train')

In [ ]:
im, gt = dataset_train[2]
print(gt.unique())
im

In [ ]:
fig, axes = plt.subplots(3,3)
gt_np = gt.numpy()
for i, ax in enumerate(axes.flatten()):
    ax.imshow((gt_np == i).astype(int))

# CLIPSeg

In [2]:
import pickle as pkl
with open('save_data.pkl', 'rb') as fp:
    save_data = pkl.load(fp)

In [3]:
state_dict = torch.load('checkpoints/clipseg_ft_VA_L_F_D_voc.pth', map_location='cpu')
clip_seg = CLIPSeg()
clip_seg.load_state_dict(state_dict['model'])

Some weights of CLIPSegForImageSegmentation were not initialized from the model checkpoint at CIDAS/clipseg-rd64-refined and are newly initialized: ['non_object_embedding', 'text_adapter.fc.0.weight', 'text_adapter.fc.2.weight', 'tunable_linear.weight', 'visual_adapter.fc.0.weight', 'visual_adapter.fc.2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [4]:
from clipseg_model import PARTS_NAMES, BASE_PARTS_NAMES
len(PARTS_NAMES), len(BASE_PARTS_NAMES)

(116, 74)

In [6]:
batch = save_data[2]
loss_dict = clip_seg(batch)

In [7]:
loss_dict

{'loss_sem_seg': tensor(0.0023, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)}

In [ ]:
a, b = torch.randn(8,117,352,352), torch.randn(8,117,352,352)

In [ ]:
F.binary_cross_entropy_with_logits(a.permute(0,2,3,1), b.permute(0,2,3,1), pos_weight=torch.ones(117))

In [ ]:
import torch.nn.functional as F

a = torch.arange(10).unsqueeze(0).unsqueeze(0).expand(3,10,-1)
m = F.one_hot(a, num_classes=10)

In [ ]:
m[0, ..., 0]

In [ ]:
mask.shape

In [ ]:
clip_seg.train_text_encoding['input_ids'].shape

In [ ]:
img_dir = 'datasets/CUB_200_2011/images/001.Black_footed_Albatross/'
fn = os.listdir('datasets/CUB_200_2011/images/001.Black_footed_Albatross/')[0]

In [ ]:
clip_seg.test_class_texts[11:20]

In [ ]:
clip_seg.eval()
with torch.no_grad():
    inputs = {'file_name': os.path.join(img_dir, fn),'sem_seg': torch.tensor([2])}
    out = clip_seg.inference([inputs])

In [ ]:
out[0]['sem_seg'].shape

In [ ]:
out[0]['outputs'].logits.shape

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(3,3)
for ax, score_map, part_name in zip(axes.flatten(), out[0]['sem_seg'].numpy(), clip_seg.part_texts):
    ax.imshow(score_map)
    ax.title.set_text(part_name)

In [ ]:
Image.open(os.path.join(img_dir, fn))

In [ ]:
Image.open(os.path.join(img_dir, fn)).size

In [ ]:
from clipseg_model import OBJ_CLASS_NAMES, PARTS_NAMES, BASE_PARTS_NAMES
len(OBJ_CLASS_NAMES), len(PARTS_NAMES), len(BASE_PARTS_NAMES)

In [ ]:
from PIL import Image
import numpy as np

In [ ]:
parts = Image.open('datasets/PASCUB/PASCAL_Parts/parts_test/2008_000339_3.png')
# 'datasets/PASCUB/PASCAL_Parts/images_test/2008_000339_3.png'
parts = np.array(parts)

In [ ]:
np.unique(parts)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, axes = plt.subplots(4, 3)
for i, ax in zip(range(11), axes.flatten()):
    ax.imshow((parts == i).astype(int))
    ax.axis('off')

In [ ]:
{0: 'background',
 1: 'head',
 2: 'beak/bill',
 3: 'tail',
 4: 'wing',
 5: 'wing',
 6: 'leg',
 7: 'leg',
 8: 'eye',
 9: 'eye',
 10: 'torso'}

In [ ]:
a = torch.tensor(np.array(Image.open('datasets/CUB_Parts/parts_val/Cliff_Swallow_0042_133646.png')), dtype=torch.long)

In [ ]:
a[a == 5] = 4
a[a == 9] = 8

In [ ]:
b, c = a.unique(return_inverse=True)

In [ ]:
a.shape, c.shape

In [ ]:
c.unique()